In [2]:
from config import *

# Prompt Templates

### Prompt Template

In [3]:
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="cats")

'Tell me a funny joke about cats.'

In [ ]:
# a template withou
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a joke"
)
prompt_template.format()

In [6]:
# for additional validation, specify input_variables explicitly

from langchain import PromptTemplate

valid_prompt = PromptTemplate(
    template="Tell me a {adjective} joke about {content}.",
    input_variables=["adjective", "content"]
)

In [ ]:
# for additional validation, specify input_variables explicitly

from langchain import PromptTemplate

invalid_prompt = PromptTemplate(
    template="Tell me a {adjective} joke about {content}.",
    input_variables=["adjective"]
)

### Chat prompt template

In [9]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ('system', "You are a helpful AI bot. Your name is {name}."),
    ('human','Hello, how are you doing?'),
    ('ai', 'I am doing well, how are you?'),
    ('human', '{user_input}'),
])

messages = template.format_messages(
    name='Bob',
    user_input='What is your name?'
)

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=("You are a helpful assistant that re-writes the user's text to "
            "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
llm(template.format_messages(text='i dont like eating tasty tings.'))

AIMessage(content='I absolutely adore indulging in delicious treats!', additional_kwargs={}, example=False)

## Connecting to a Feature Store

### Feast

# Custom prompt template

### Creating a custom prompt template

In [16]:
import inspect
def get_source_code(function_name):
    return inspect.getsource(function_name)

In [17]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator

In [18]:
PROMPT = """\
    Given the function name and source code, generate an English language explanation of the function.
    Function Name: {function_name}
    Source Code: 
    {source_code}
    Expalanation:
    """

In [20]:
class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes in the fucntion name as input, and formats the prompt template to provide the source code of the function."""
    
    @validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input variables are correct."""
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input variable.")
        return v
    
    def format(self, **kwargs) -> str:
        # Get the source code of the function
        source_code  = get_source_code(kwargs["function_name"])
        
        # Generate the prompt to be sent to the language model
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__, source_code=source_code
        )
        return prompt
    
    def _prompt_type(self) -> str:
        return super()._prompt_type() + "function_explainer"

### Use the custom prompt template


In [22]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

# Generate a prompt for the function "get_source_code"ArithmeticError
prompt = fn_explainer.format(function_name=get_source_code)

In [24]:
print(prompt)

    Given the function name and source code, generate an English language explanation of the function.
    Function Name: get_source_code
    Source Code: 
    def get_source_code(function_name):
    return inspect.getsource(function_name)

    Expalanation:
    


## Few-shot prompt templates

In [25]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [27]:
examples = [
    {
        "question": "who lived longer, Muhammad Ali or Alan Turing?",
        "answer": 
            """
            Are follow up questions needed here: Yes.
            Follow up: How old was Muhammad Ali when he died?
            Intermediate answer: Muhammad Ali was 74 years old when he died.
            Follow up: How old was Alan Turing when he died?
            Intermediate answer: Alan Turing was 41 years old when he died.
            So the final answer is: Muhammad Ali
            """
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer":
            """
            Are the follow up questions needed here: Yes.
            Follow up: who was the founder of craigslist?
            Intermediate answer: Craig Newmark was the founder of craigslist.
            Follow up: When was Craig Newmark born?
            Intermediate answer: Craig Newmark was born on December 6, 1952.
            So the final answer is: December 6, 1952
            """
    },
    {
    "question": "Who was the maternal grandfather of George Washington?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
"""
  },
  {
    "question": "Are both the directors of Jaws and Casino Royale from the same country?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
"""
  }
]

### Create a formatter for the few shot examples

In [28]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\nAnswer: {answer}\n\n")
print(example_prompt.format(**examples[0]))

Question: who lived longer, Muhammad Ali or Alan Turing?
Answer: 
            Are follow up questions needed here: Yes.
            Follow up: How old was Muhammad Ali when he died?
            Intermediate answer: Muhammad Ali was 74 years old when he died.
            Follow up: How old was Alan Turing when he died?
            Intermediate answer: Alan Turing was 41 years old when he died.
            So the final answer is: Muhammad Ali
            




### Feed examples and formatter to FewShotPromptTemplate 

In [29]:
prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt=example_prompt,
    suffix="Question:{input}\nAnswer:",
    input_variables=["input"],
)

In [31]:
print(prompt.format(input="Who is the President of Pakistan?"))

Question: who lived longer, Muhammad Ali or Alan Turing?
Answer: 
            Are follow up questions needed here: Yes.
            Follow up: How old was Muhammad Ali when he died?
            Intermediate answer: Muhammad Ali was 74 years old when he died.
            Follow up: How old was Alan Turing when he died?
            Intermediate answer: Alan Turing was 41 years old when he died.
            So the final answer is: Muhammad Ali
            



Question: When was the founder of craigslist born?
Answer: 
            Are the follow up questions needed here: Yes.
            Follow up: who was the founder of craigslist?
            Intermediate answer: Craig Newmark was the founder of craigslist.
            Follow up: When was Craig Newmark born?
            Intermediate answer: Craig Newmark was born on December 6, 1952.
            So the final answer is: December 6, 1952
            



Question: Who was the maternal grandfather of George Washington?
Answer: 
Are follow up

### Using an example selector

In [32]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [35]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma,
    k=2)

In [39]:
# Select the most similar example to the input.

question = "Who was the father of craigslist founder?"
selected_examples = example_selector.select_examples({'question': question})
print(selected_examples)
for example in selected_examples:
    print(example['question'])
    print(example['answer'])
    print()

[{'question': 'When was the founder of craigslist born?', 'answer': '\n            Are the follow up questions needed here: Yes.\n            Follow up: who was the founder of craigslist?\n            Intermediate answer: Craig Newmark was the founder of craigslist.\n            Follow up: When was Craig Newmark born?\n            Intermediate answer: Craig Newmark was born on December 6, 1952.\n            So the final answer is: December 6, 1952\n            '}, {'question': 'Who was the maternal grandfather of George Washington?', 'answer': '\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of George Washington was Mary Ball Washington.\nFollow up: Who was the father of Mary Ball Washington?\nIntermediate answer: The father of Mary Ball Washington was Joseph Ball.\nSo the final answer is: Joseph Ball\n'}]
When was the founder of craigslist born?

            Are the follow up questions needed here: Yes.


### Feed example selector into FewShotPromptTemplate

In [42]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:{question}\nAnswer:",
    input_variables=["question"],
)

print(prompt.format(question="Who is a fighter?"))

Question: Are both the directors of Jaws and Casino Royale from the same country?
Answer: 
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No




Question: who lived longer, Muhammad Ali or Alan Turing?
Answer: 
            Are follow up questions needed here: Yes.
            Follow up: How old was Muhammad Ali when he died?
            Intermediate answer: Muhammad Ali was 74 years old when he died.
            Follow up: How old was Alan Turing when he died?
            Intermediate answer: Alan Turing was 41 years old when he died.
            So the final answer is: Muhammad Ali
         

## Few shot examples for chat models